# Part 1: Theory of Fine-Tuning

## Concept Check (Multiple Choice Questions)
What is the main benefit of fine-tuning an LLM?

A) It improves the model’s speed.

**B) It customizes the model for specific tasks or domains.**

C) It eliminates the need for high-quality datasets.

D) It prevents overfitting entirely.


Which of the following describes "catastrophic forgetting"?

A) When the model forgets its pre-training data during inference.

**B) When the model loses its generalization ability after excessive fine-tuning on a specific task.**

C) When the model produces irrelevant outputs due to overfitting.

D) When the model fails to save fine-tuned weights.

## Application Task

1. Write a 150–200 word explanation of transfer learning using a real-world analogy. Use examples from any domain (e.g., healthcare, legal, e-commerce).

  A: Transfer learning is the process by which a pre-trained model is adapted for a different or related task. Fine tuning LLMs is the type of transfer learning in this context. Let's use an example of an AI chatbot that answers customer questions about a product based on the product reviews and description. The initial model would be a pre-trained LLM. Then, it would undergo the process of fine tuning, where the input would be the customer reviews and product description and the output would be answers to questions about the product. For an air fryer, a user might ask: "Do customers say this is reliable long term?" and the chatbot might answer: "Yes, customers frequently report the air fryer working after well over a year."
2. Provide an example dataset structure for a fine-tuning task of your choice. Label and clean your dataset to match the requirements for the task.

  See imdb dataset preprocessing below

# Part 2: Practical Fine-Tuning Session

## 1. Environment Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
!pip install datasets
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00


In [4]:
# Check for GPU availability
print(torch.cuda.is_available()) # Should return True

True


## 2. Load and Preprocess Data

In [5]:
model_name = "distilbert-base-uncased"
model_checkpoint_dir = "/content/drive/MyDrive/model_checkpoint"

# if training from start:
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# if opening saved model:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint_dir)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_dir)

In [6]:
dataset = load_dataset("imdb")
def preprocess_function(examples):
  return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## 3. Fine-Tune the Model

Define Training Arguments

In [7]:
training_args = TrainingArguments(
  output_dir="./results",
  save_steps=500,
  save_total_limit=3,
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  num_train_epochs=3, weight_decay=0.01,
)
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset["train"],
  eval_dataset=tokenized_dataset["test"],
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Train the Model

In [8]:
# if training from start:
# trainer.train()
# if opening saved model:
trainer.train(resume_from_checkpoint=True)

ValueError: No valid checkpoint found in output directory (./results)

Save the Fine-Tuned Model

In [9]:
model.save_pretrained(model_checkpoint_dir)
tokenizer.save_pretrained(model_checkpoint_dir)
!cp -r ./results /content/drive/MyDrive/results

## 4. Save and Evaluate

In [10]:
results = trainer.evaluate()
print(results)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kcauwenb (kcauwenb-uc-san-diego-health) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.2828763723373413, 'eval_model_preparation_time': 0.0015, 'eval_runtime': 355.1949, 'eval_samples_per_second': 70.384, 'eval_steps_per_second': 8.798}


Detailed metrics with sklearn

In [11]:
predictions = trainer.predict(tokenized_dataset["test"])
y_pred = predictions.predictions.argmax(axis=1)
y_true = tokenized_dataset["test"]["label"]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93     12500
           1       0.93      0.94      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



## Reflection
Summarize the key challenges you faced during the fine-tuning process and how you addressed them.
Provide suggestions for improving the model’s performance if the accuracy was below 90%.

Challenges I faced:
- This was my first time using Colab for a long time since I haven't used GPU to run trainings for a while so I had to relearn the logistics of it.
- The training took a long time and I had to figure out how to save my results. I often lost my data. So I learned the hard way to save my results to google drive just in case the runtime disconnected.

The model performance seemed ok, the accuracy was over 90%. If I wanted to improve it though, I would increase the batch size and number of epochs.